<a href="https://colab.research.google.com/github/francobarrionuevoenv21/codeforenviroprojects/blob/testGEE/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api

In [ ]:
!pip install geemap

In [15]:
import geemap
import ee
import geopandas as gpd
import pandas as pd

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [5]:
# Code to read csv file into Colaboratory:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
def getCoordsSHP(gdfshpFile):
  '''
  This function takes a polygon vector layer file passed as a GeoDataFrame. Then
  it gets all longitude and latitude values and return it as numpy arrays.
  '''

  gdfiterf = gdfshpFile.iterfeatures()
  tupleFile = next(gdfiterf)['geometry']['coordinates']

  #arrayCoords = np.array([list(tupleFile[0][i]) for i in range(len(tupleFile[0]))])

  return tupleFile

In [7]:
def FileDataDf(dictFiles, i, source):

  if source == 'gdrive': # Files hosted in Google Drive.
    FilesPath = list(dictFiles.values())[i]
    idPath = FilesPath.split('/', 6)
    downloadedFiles = drive.CreateFile({'id':idPath[5]})
    downloadedFiles.GetContentFile(list(dictFiles.keys())[i])
    dfData = pd.read_csv(list(dictFiles.keys())[i])

  elif source == 'other': # Files imported from device or Colab files
    dfData = pd.read_csv(list(dictFiles.values())[i])

  return dfData

In [8]:
# Load the shapefile
shapefile_path = '/content/DeltaParana.shp'
gdf = gpd.read_file(shapefile_path)

# Extract the polygon
polygon = gdf.geometry.values[0]

# Print the polygon
print(polygon)


POLYGON Z ((-60.256769 -33.268629 0, -60.257571 -33.26818 0, -60.258462 -33.268045 0, -60.259577 -33.267517 0, -60.262146 -33.266485 0, -60.263861 -33.265576 0, -60.265684 -33.264619 0, -60.266487 -33.264372 0, -60.26707 -33.26391 0, -60.2686 -33.262831 0, -60.269695 -33.262337 0, -60.270715 -33.261598 0, -60.271736 -33.261042 0, -60.272393 -33.260795 0, -60.273049 -33.260302 0, -60.273705 -33.260116 0, -60.277272 -33.261 0, -60.287335 -33.253839 0, -60.291571 -33.25282 0, -60.293759 -33.251925 0, -60.295775 -33.246619 0, -60.299672 -33.245559 0, -60.302706 -33.244483 0, -60.30602 -33.242575 0, -60.308626 -33.240194 0, -60.311156 -33.236566 0, -60.313336 -33.233652 0, -60.316507 -33.231447 0, -60.319114 -33.229719 0, -60.321578 -33.227457 0, -60.323832 -33.22567 0, -60.327637 -33.223167 0, -60.329678 -33.22138 0, -60.333057 -33.218403 0, -60.33899 -33.218387 0, -60.344843 -33.216115 0, -60.347022 -33.213378 0, -60.348006 -33.212247 0, -60.350547 -33.211825 0, -60.353157 -33.211105 0, -

In [ ]:
type(polygon_layer)

NoneType

In [41]:
StudyArea = gpd.read_file('/content/DeltaParana.shp')
# getCoordsSHP(StudyArea)[0]

In [42]:
# Sample tuple of tuples
tuple_of_tuples = getCoordsSHP(StudyArea)[0]

# Convert to a list of lists with only the first 2 elements
list_of_lists = [list(t[:2]) for t in tuple_of_tuples]

# Print the result
type(list_of_lists[0][0])


float

In [43]:
coordinates = list_of_lists

# Create an ee.Geometry.Polygon object
polygon = ee.Geometry.Polygon(coordinates)

# Print the polygon
#print(polygon)

In [52]:
import geemap
import geopandas as gpd

# Load the shapefile
shapefile_path = '/content/DeltaParana.shp'
gdf = gpd.read_file(shapefile_path)

# Create a Map instance
Map = geemap.Map()


# Add the polygon layer with visualization parameters
#Map.addLayer(polygon, vis_params, 'Polygon')

# Display the map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [56]:
# Define the visualization options
vis_params = {
    'color': 'FF0000',   # Red color for the outline
    'fillColor': 'transparent',   # Black color for the fill
    'fillOpacity': 1,  # Fully transparent fill
    #'weight': 2   # Thickness of the outline
}

# Create an ee.FeatureCollection from the polygon
# fc = ee.FeatureCollection(polygon)

# Add the polygon layer with custom style function
Map.addLayer(polygon, vis_params, 'Polygon')
Map.addLayer(fc, {}, 'Points')

# Set the map center to the polygon
Map.centerObject(polygon, 7)

In [18]:
dictFilesPath ={
    "1.csv" : "https://drive.google.com/file/d/1YdDj72tl-rnQpzIaZSyd6see2aoPMewg/view?usp=share_link",
    }

dfHSArgDelta = FileDataDf(dictFilesPath, 0, source = 'gdrive')

gdfHSArgDelta = gpd.GeoDataFrame(
    dfHSArgDelta,
    geometry=gpd.points_from_xy(dfHSArgDelta['longitude'], dfHSArgDelta['latitude']),
    crs='epsg:4326')

In [47]:
lat_long_list = gdfHSArgDelta.head(50)[['longitude', 'latitude']].values.tolist()
points = ee.Geometry.LineString(lat_long_list)
Map.addLayer(points, {}, 'Polygon')
#type(lat_long_list[0][0])

In [49]:
points = gdfHSArgDelta.head(50)[['longitude', 'latitude']].values.tolist()

# Create a feature collection from the points
fc = geemap.geojson_to_ee({'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': point}} for point in points]})

# Add the feature collection to the map
Map.addLayer(fc, {}, 'Points')

In [ ]:
lat_long_list

De aca para abajo adsadsad

In [ ]:
import geemap
import geopandas as gpd

# Load the shapefile
shapefile_path = '/content/DeltaParana.shp'
gdf = gpd.read_file(shapefile_path)

# Convert the GeoDataFrame to an ee.FeatureCollection
fc = geemap.geopandas_to_ee(gdf)

# Create a Map instance
Map = geemap.Map()

# Define the visualization options
vis_params = {
    'color': '000000',
    'fillColor': '00000000',  # Transparent fill color
    'width': 2,
}

# Add the polygon layer with the visualization parameters
Map.addLayer(fc, vis_params, 'Delta')

# Center the map on the polygon
Map.centerObject(fc, 7)

# Display the map
Map


Could not convert the geojson to ee.Geometry()


Exception: ignored

In [ ]:

gdf = gpd.read_file(
    "https://raw.githubusercontent.com/giswqs/data/main/us/us_states.geojson"
)
gdf.head()

Map = geemap.Map()

fc = geemap.geopandas_to_ee(gdf)

vis_params = {
    'color': '000000',
    'fillColor': 'transparent',  # Transparent fill color
    'width': 2,
}

Map.addLayer(fc, vis_params, "geopandas to ee")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# the regulars
import pandas as pd
import geopandas as gpd
import numpy as np

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

In [ ]:
# Coordenadas 2018: -32.81910507118151, -60.42389548016447
# in decimal degrees
lat =  -32.81910507118151
lon = -60.42389548016447


# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# Landsat-8 takes 8 days to come back to the same place

# start date of range to filter for
start_date = '2020-08-01'

# end date
end_date = '2020-09-10'

# get the satellite data
Landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

# put the images in a list
landsat_list = Landsat8.toList(Landsat8.size());

for i in range(Landsat8.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

    print(date, cloud)

2020-08-04 1.18
2020-08-20 0.42
2020-09-05 23.4
2020-08-11 5.27
2020-08-27 21.47


In [ ]:

parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 600, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

display(Image(url = ee.Image(landsat_list.get(1)).getThumbUrl(parameters)))

In [ ]:
#Landsat8.size()
#print(landsat_list)

In [ ]:
print(ee.Image(landsat_list.get(1)).bandNames().getInfo())

print(ee.Image(landsat_list.get(1)).select('SR_B1').projection().getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
{'type': 'Projection', 'crs': 'EPSG:32620', 'transform': [30, 0, 528585, 0, -30, -3554085]}


In [ ]:
# Se carga la imagen .shp correspondiente a los ROIs a partir del directorio
#rois_gdp = gpd.read_file('D:\TELEDETECCION\Proyecto_incendios_Delta\Muestras_2018-08-22 32620\Muestras_2018-08-22_32620.shp')#.set_index('id') #Levanta archivo shp
rois_gdp = gpd.read_file('D:\TELEDETECCION\Proyecto_incendios_Delta\Muestras_2018-08-22.shp')#.set_index('id') #Levanta archivo shp

# print(rois_gdp.geometry.values[0])

#geometry = rois_gdp.geometry.values[0]
rois_gdp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
rois_gdp_exp = rois_gdp.explode(index_parts = False)

In [ ]:
rois_gdp_exp.geometry[0]

TypeError: 'Polygon' object is not subscriptable

In [ ]:
#https://stackoverflow.com/questions/4741243/how-to-pick-just-one-item-from-a-generator
rois_gdp_iterf = rois_gdp.iterfeatures()
geom_data = next(rois_gdp_iterf)['geometry']['coordinates']

In [ ]:
list_lon = []
list_lat = []

for i in range(len(geom_data[0])):
    list_lon.append(geom_data[0][i][0])
    list_lat.append(geom_data[0][i][1])

In [ ]:
list_coord = []

for i in range(len(geom_data[0])):
    list_coord.append(list(geom_data[0][i]))

list_coord

[[-60.43470604358266, -32.834308101686894],
 [-60.36500181902507, -32.834154378659115],
 [-60.359696248126994, -32.95597310931026],
 [-60.43049817700832, -32.956740665506416],
 [-60.43470604358266, -32.834308101686894]]

In [ ]:
geom_clip = ee.Geometry.Polygon(list_coord)

In [ ]:
geom_clip

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -60.43470604358266,
              -32.834308101686894
            ],
            [
              -60.36500181902507,
              -32.834154378659115
            ],
            [
              -60.359696248126994,
              -32.95597310931026
            ],
            [
              -60.43049817700832,
              -32.956740665506416
            ],
            [
              -60.43470604358266,
              -32.834308101686894
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [ ]:
#ee.Image(landsat_list.get(1)).clip(geometry)

display(Image(url = ee.Image(landsat_list.get(1)).clip(geom_clip).getThumbUrl(parameters)))

Export image

References:
https://eamena.org/sites/default/files/eamena/documents/media/gis_4_-_introduction_to_gee_and_sentinel-2_imagery-compressed.pdf
https://gis.stackexchange.com/questions/300344/exported-bands-must-have-compatible-data-types-found-inconsistent-types-uint16
https://gis.stackexchange.com/questions/347340/export-google-earth-engine-landsat-image-to-google-drive-not-working-in-python-a

In [ ]:
select_bands = ee.Image(landsat_list.get(1)).bandNames().getInfo()[0:6]
select_bands

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6']

In [ ]:
image_bands = ee.Image(landsat_list.get(1)).select(select_bands)

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image = image_bands,
    region = geom_clip,
    #description = 'test01'
)

In [ ]:
export_task.start()

In [ ]:
export_task.status()

{'state': 'READY',
 'description': 'myExportImageTask',
 'creation_timestamp_ms': 1672542935978,
 'update_timestamp_ms': 1672542935978,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'SOGVC6ZNDRHMGGGMWYYPFWHM',
 'name': 'projects/earthengine-legacy/operations/SOGVC6ZNDRHMGGGMWYYPFWHM'}